In [23]:
from transformers import pipeline
import json

In [24]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

context = "Manuel has created RuPERTa-base with the support of HF-Transformers and Google"
answer = "Manuel"

get_question(answer, context)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/Users/joshuamin/miniconda3/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:998: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


'<pad> question: Who created the RuPERTa-base?</s>'

In [25]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast

hfmodel = T5ForConditionalGeneration.from_pretrained("ThomasSimonini/t5-end2end-question-generation")
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

text= '''Let's begin by recalling that the control unit consists of three parts: a 
high-level FSM that controls instruction processing, a program counter~(PC)
register that holds the address of the next instruction to be executed,
and an instruction register~(IR) that holds the current instruction as
it executes.'''

def run_model(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = "generate questions: " + input_string + " </s>"
  input_ids = tokenizer.encode(input_string, return_tensors="pt")
  res = hfmodel.generate(input_ids, **generator_args)
  output = tokenizer.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output
  
run_model(text)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


[['What controls instruction processing? What holds the address of the next instruction to be executed?']]

In [26]:
f = open('GPT-3.json')
data = json.load(f)

In [28]:
# fine_tuned_data = []

# tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
# model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

# for row in data:
#     one_data = {}

#     context = row['positive_ctxs']['text']
#     answer = row['answers']

#     question = get_question(answer, context)

#     one_data['positive_ctxs'] = {}
#     one_data['questions'] = question
#     one_data['answers'] = answer
#     one_data['positive_ctxs']['title'] = row['positive_ctxs']['title']
#     one_data['positive_ctxs']['text'] = context

#     fine_tuned_data.append(one_data)


In [29]:
# with open('Fine_Tuned_Data.json', 'w', encoding='utf-8') as f:
#     json.dump(ftd, f, ensure_ascii=False, indent=4)

In [20]:
f = open('Fine_Tuned_Data.json')
ftd = json.load(f)

In [33]:
end = []

for row in data:
    one_data = {}

    context = row['positive_ctxs']['text']

    question = run_model(context)

    one_data['positive_ctxs'] = {}
    one_data['questions'] = question
    one_data['answers'] = row['answers']
    one_data['positive_ctxs']['title'] = row['positive_ctxs']['title']
    one_data['positive_ctxs']['text'] = context

    end.append(one_data)

In [35]:
with open('end2end.json', 'w', encoding='utf-8') as f:
    json.dump(ftd, f, ensure_ascii=False, indent=4)